In [2]:
import requests
import time
import hashlib
from decouple import config
import pandas as pd
import json
from datetime import datetime

Geração do timestamp

In [3]:
timestamp_atual = str(time.time())

Declaração das chaves

In [4]:
pubk = config('pubk')
pvk = config('pvk')

Criação do md5_hash

In [5]:
md5_hash = hashlib.md5()
md5_hash.update(bytes(timestamp_atual, 'utf-8'))
md5_hash.update(bytes(pvk, 'utf-8'))
md5_hash.update(bytes(pubk, 'utf-8'))
md5 = md5_hash.hexdigest()

URL

In [ ]:
URL = f"https://gateway.marvel.com/v1/public/comics?apikey={pubk}&hash={md5}&ts={timestamp_atual}&limit=50"

Função para salvar os dados em formato JSON

In [ ]:
"""usar timestamp para o controle de data"""

In [14]:
try:
    with open("contador.json", "r") as f:
        contador_requisicoes = json.load(f)
except FileNotFoundError:
    contador_requisicoes = 0

# Função que recebe a URL e o nome do arquivo como parâmetro
def salvar_dados_api(URL, nome_arquivo):
    global contador_requisicoes
    
    try:
        # Faz a requisição da API e armazena em uma variável
        dados = requests.get(URL).json()
        
        # Incrementa o contador
        contador_requisicoes += 1
        
        # Salva o contador no arquivo
        with open("contador.json", "w") as f:
            json.dump(contador_requisicoes, f)
        
        # Criando um dataframe com o variável "dados"
        df = pd.DataFrame(dados)
        
        # Criação da coluna "data_download" para marcar a hora que foi realizada requisição
        df['data_download'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Salvar em JSON
        df.to_json(f'{nome_arquivo}.json', orient='records', indent=4)
        
        print(f"Total de requisições feitas: {contador_requisicoes}")
        print(f"Dados salvos com sucesso! Total de registros: {len(df)}")
        return df
    
    # Tratamento de erros 
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")
        


# Requisição 
dados_salvos = salvar_dados_api(URL, "characters")

Total de requisições feitas: 3
Dados salvos com sucesso! Total de registros: 5
